!pip install pandas numpy scipy matplotlib seaborn lxml tqdm validators pprint

In [3]:
"""
target website uri - https://dentalstall.com/shop/
for each other target uri [recurrent pages] - https://dentalstall.com/shop/page/1

paginator class
"""

'\ntarget website uri - https://dentalstall.com/shop/\nfor each other target uri [recurrent pages] - https://dentalstall.com/shop/page/1\n\npaginator class\n'

In [485]:
import string
import logging
import pandas as pd
from lxml import html
from bs4 import BeautifulSoup, Tag
from urllib.request import Request, urlopen
from IPython.display import display, HTML
import random 
import pprint
import traceback

## configure the base logger settings for logging
logger = logging.getLogger(__name__)
logging.basicConfig()
logger.setLevel('DEBUG')

## setting the pd dataframe maximum width
pd.set_option('display.max_colwidth', None)


__agent_header_list = [
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; x64; fr; rv:1.9.2.13) Gecko/20101203 Firebird/3.6.13',
    'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows NT 5.2; RW; rv:7.0a1) Gecko/20091211 SeaMonkey/9.23a1pre'
]

## defined the Agent Header identifier for secure protocol
AGENT_HEADER = {'User-Agent': random.choice(__agent_header_list), 'X-Requested-With': 'XMLHttpRequest'}
logger.info(AGENT_HEADER)

__init_website_uri = "https://dentalstall.com/shop/"

def __perform_query_search(*, search_href: str) -> typing.Union[None, Tag]:
    ## handling to scrape persistive information from INIT_WEBSITE_URI
    soup_query_response: typing.Union[None, Tag] = None
    try:
        if __init_request := Request(url=search_href, headers=AGENT_HEADER): # ramdomly chosen header agent to perform query
            scrape_authorisation = urlopen(__init_request).read() # performing web authorisation for request uri
            soup_query_response = BeautifulSoup(scrape_authorisation.decode('utf-8'), 'html.parser') # concieve the HTML format response on scraped website
        
        ## retry handlin' on search web-query post N seconds
        else:
            soup_query_response = None

    except Exception:
        logger.error(traceback.format_exc())
        return None

    else:
        return soup_query_response
        
"""
need to perform query filter on displayed product on init_query website
product-metadata -> product_name: str, published_price: union[int, str], associated_images: List['uri': str]

restricted options for querying:
    - limit the scrape processing by N: no of page to-be scraped
    - query filter by addon - proxy string param {presumption: product_tag, updated_by, by_review_quantifier, price_cap, discount_offered}
"""
soup_query_response = __perform_query_search(search_href=__init_website_uri)
soup_query_response.prettify


INFO:__main__:{'User-Agent': 'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16', 'X-Requested-With': 'XMLHttpRequest'}


<bound method Tag.prettify of <!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://dentalstall.com/xmlrpc.php" rel="pingback"/>
<link as="font" crossorigin="" href="https://dentalstall.com/wp-content/themes/martfury/fonts/linearicons.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://dentalstall.com/wp-content/themes/martfury/fonts/eleganticons.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://dentalstall.com/wp-content/themes/martfury/fonts/ionicons.woff2" rel="preload" type="font/woff2"/>
<title>Shop – Dentalstall</title><link as="style" href="https://fonts.googleapis.com/css2?family=Archivo%3Awght%40300%3B400%3B500%3B600%3B700&amp;ver=6.6.2&amp;display=swap" rel="preload"><link href="https://fonts.googleapis.com/css2?family=Archivo%3Awght%40300%3B400%3B500%3B600%3B700&amp;ver=6.6.2&amp;display=swap" media="pri

In [489]:
from collections import OrderedDict
import typing
import uuid
import traceback
import json

# soup_query_response.xpath("//div[starts-with(text(),'Product ')]")
# __product_content_attr = soup_query_response.xpath("//div[@class='mf-shop-content' and @id='mf-shop-content']//child::ul")[0]
# soup_query_response = html.fromstring(scrape_authorisation)
# print(html.tostring(soup_query_response))

## implementing the lxml.html parser handlin
# for _attr in __product_content_attr[:1]:
#     # print((_attr.find_class('product-inner clearfix')[0]))
#     for _product_div in _attr.find_class('product-inner clearfix'):
#         # # print(_div1.find_class(class_name='mf-product-thumbnail'))
#         v = _product_div.xpath("//div[@class='mf-product-thumbnail']//img/@srcset")
#         print(v)
#         break

__pg_product_mapper: typing.List = list() 
limit_page_counter: int = 10 ## default value for limit page counter[need to set as env]
paginator_scraping: bool = False

columns = ["product_identifier", "page_id", "product_description", "published_price", "discounted_price",  "linked_images"]

def __perform_restricted_src_scraping(*, soup_query_response: Tag, __pg_product_mapper: typing.List[typing.Dict], 
                                      page_idx: typing.Union[None, int] = 1, __paginator_index: typing.Union[None, typing.List] = None):
    
    global limit_page_counter
    if not soup_query_response:
        page_idx = page_idx + 1
        __followed_page_href = "https://dentalstall.com/shop/page/{0}".format(page_idx)
        print(__followed_page_href)
        
        if __recurring_query_resp := __followed_page_href and __perform_query_search(search_href=__followed_page_href):
            return __perform_restricted_src_scraping(
                        soup_query_response=__recurring_query_resp,
                        __pg_product_mapper=__pg_product_mapper,
                        page_idx=page_idx
                    )
        
    # for _, page_content in enumerate(soup_query_response.findAll("div",class_="mf-shop-content")):
    ## keep track on website/uri paginator identifier to scrape products metadata by page-limit
    if __content_paginator := not __paginator_index and soup_query_response.select("nav ul li", class_="woocommerce-paginator"):
        __paginator_index = sorted([int(_iter.get_text()) for _iter in __content_paginator if _iter.get_text().isnumeric()], reverse=False)

    ## perform the scraper limit handlin' on target uri address
    if __paginator_index and limit_page_counter:
        if not limit_page_counter in range(__paginator_index[0], __paginator_index[-1] + 1):
            limit_page_counter = __paginator_index[-1] ## setting the limit_page_counter index to last paginator val for uri
            
    else:
        limit_page_counter = 1
    # print(__paginator_index, " ", page_idx)

    print(limit_page_counter)
    while page_idx <= limit_page_counter - 1:
        ## caution handlin' to scrape all enlist products pagewise,
        ## and deduce multi-FOR handlin -> WHILE operator sujective
        if scraped_product_info := soup_query_response.findAll("div", class_=["product-inner clearfix"]):
            __product_iterator: typing.Union[None, typing.Iterator] = iter(scraped_product_info)
    
        else:
            __product_iterator = None
    
        try:
            ## scrape the identified metadata for products enlisted on target uri
            _product: typing.Union[None, Tag] = None
            while _product := __product_iterator and next(
                    __product_iterator, None
                ):
                __product_hash_mapper, __meta_images = OrderedDict(list()), list()
                # print(_product)
    
                ## regex handlin' to omit the noisy followed characters in product images link
                import re ## contextual handlin' import
                regex_qualifier = r"\s*\d+w$"
                _product_attrs = _product.select("img")[0].attrs
                if __fetch_assoc_img_links := _product_attrs.get("data-lazy-srcset") if "data-lazy-srcset" in _product_attrs else _product_attrs.get("data-lazy-src"):
                    __meta_images.extend([link.split(re.search(regex_qualifier, link)[0])[0].strip() if re.search(regex_qualifier, link) else link.strip() for link in __fetch_assoc_img_links.split(',')])
                    # print(__meta_images)
                        
                else:
                    print(_product.select("img")[0].attrs)
                    
                __product_hash_mapper["product_identifier"] = str(uuid.uuid1())
                __product_hash_mapper["page_id"] = page_idx
                __product_hash_mapper["product_description"] = _product.select("h2")[0].get_text().strip()
    
                ## if not discount offered on product
                if product_price := _product.select("del span bdi"):
                    __product_hash_mapper["published_price"] = product_price[0].get_text()
    
                elif product_price := _product.select("span bdi"):
                    __product_hash_mapper["published_price"] = product_price[0].get_text()
                    
                else:
                    __product_hash_mapper["published_price"] = 0.00
                    
                ## if products discounted value exists
                if discount_price := _product.select("ins span"):
                    __product_hash_mapper["discounted_price"] = discount_price[0].get_text()
        
                else:
                    __product_hash_mapper["discounted_price"] = __product_hash_mapper["published_price"]
                
                __product_hash_mapper["linked_images"] = __meta_images
                __pg_product_mapper.append(__product_hash_mapper)

            page_idx = page_idx + 1
            __followed_page_href = "https://dentalstall.com/shop/page/{0}".format(page_idx)
            print(__followed_page_href)
            
            if __recurring_query_resp := __followed_page_href and __perform_query_search(search_href=__followed_page_href):
                return __perform_restricted_src_scraping(
                            soup_query_response=__recurring_query_resp,
                            __pg_product_mapper=__pg_product_mapper,
                            page_idx=page_idx
                        )

            ## if execption intercepted or request uri not found, skip current page and icrement page_idx
            else:
                page_idx += 1
                __followed_page_href = "https://dentalstall.com/shop/page/{0}".format(page_idx)
                return __perform_restricted_src_scraping(
                            soup_query_response=__recurring_query_resp,
                            __pg_product_mapper=__pg_product_mapper,
                            page_idx=page_idx
                        )
                
        except StopIteration:
            logger.error(traceback.format_exe())
            
        return __pg_product_mapper
        
    print()

__perform_restricted_src_scraping(
    soup_query_response=soup_query_response,
    __pg_product_mapper=__pg_product_mapper,
)

# print(__pg_product_mapper)
prod_metadata_df = pd.DataFrame(__pg_product_mapper, columns=columns)
prod_metadata_df.reset_index()
#_json = json.loads(prod_metadata_df.to_json())



10
https://dentalstall.com/shop/page/2
10
https://dentalstall.com/shop/page/3
10
https://dentalstall.com/shop/page/4


ERROR:__main__:Traceback (most recent call last):
  File "/var/folders/wk/z3bldfl563x0jthyxwvxv3qh0000gn/T/ipykernel_36981/672280997.py", line 40, in __perform_query_search
    scrape_authorisation = urlopen(__init_request).read() # performing web authorisation for request uri
                           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
               ^^^^^^^^^^^^^^^^^^
  File

10
https://dentalstall.com/shop/page/5
https://dentalstall.com/shop/page/7
10
https://dentalstall.com/shop/page/8
10
https://dentalstall.com/shop/page/9
10
https://dentalstall.com/shop/page/10
10



,index,product_identifier,page_id,product_description,published_price,discounted_price,linked_images
0,0,daf98e5c-7d7c-11ef-acef-7295f59fdab3,1,1 x GDC Extraction Forceps Lo...,₹1250.00,₹850.00,"[https://dentalstall.com/wp-content/uploads/2021/11/GDC-Extraction-Forceps-Lower-Molars-86A-Standard-FX86AS-300x300.jpg, https://dentalstall.com/wp-content/uploads/2021/11/GDC-Extraction-Forceps-Lower-Molars-86A-Standard-FX86AS-100x100.jpg]"
1,1,daf9a496-7d7c-11ef-acef-7295f59fdab3,1,3A MEDES Bleaching And Night ...,₹1380.00,₹1380.00,"[https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1-300x300.jpg, https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1-100x100.jpg, https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1-600x600.jpg, https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1-150x150.jpg, https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1-768x768.jpg, https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1.jpg]"
2,2,daf9b850-7d7c-11ef-acef-7295f59fdab3,1,3m Espe Adper Easy Bond Self-...,₹2863.00,₹2365.00,"[https://dentalstall.com/wp-content/uploads/2023/03/3mespe082010-300x300.jpg, https://dentalstall.com/wp-content/uploads/2023/03/3mespe082010-100x100.jpg, https://dentalstall.com/wp-content/uploads/2023/03/3mespe082010-150x150.jpg, https://dentalstall.com/wp-content/uploads/2023/03/3mespe082010.jpg]"
3,3,daf9caac-7d7c-11ef-acef-7295f59fdab3,1,3M ESPE Adper Scotchbond Mult...,₹1195.00,₹1195.00,"[https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-300x300.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-100x100.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-600x600.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-1024x1024.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-150x150.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-768x768.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7.jpg]"
4,4,daf9e730-7d7c-11ef-acef-7295f59fdab3,1,3m Espe Adper Single Bond Uni...,₹2590.00,₹2590.00,"[https://dentalstall.com/wp-content/uploads/2021/12/3m-Espe-Adper-Single-Bond-Universal-Adhesive-300x300.jpg, https://dentalstall.com/wp-content/uploads/2021/12/3m-Espe-Adper-Single-Bond-Universal-Adhesive-100x100.jpg, https://dentalstall.com/wp-content/uploads/2021/12/3m-Espe-Adper-Single-Bond-Universal-Adhesive.jpg]"
...,...,...,...,...,...,...,...
163,163,ddc6f8a4-7d7c-11ef-acef-7295f59fdab3,9,API Orthodontic Cutter...,₹1955.00,₹1955.00,"[https://dentalstall.com/wp-content/uploads/2023/05/1._read_more_books2._get_a_new_hobby3._try_rock_climbing4._be_more_creative_1__1-300x300.jpg, https://dentalstall.com/wp-content/uploads/2023/05/1._read_more_books2._get_a_new_hobby3._try_rock_climbing4._be_more_creative_1__1-100x100.jpg, https://dentalstall.com/wp-content/uploads/2023/05/1._read_more_books2._get_a_new_hobby3._try_rock_climbing4._be_more_creative_1__1-600x600.jpg, https://dentalstall.com/wp-content/uploads/2023/05/1._read_more_books2._get_a_new_hobby3._try_rock_climbing4._be_more_creative_1__1-150x150.jpg, https://dentalstall.com/wp-content/uploads/2023/05/1._read_more_books2._get_a_new_hobby3._try_rock_climbing4._be_more_creative_1__1-768x768.jpg, https://dentalstall.com/wp-content/uploads/2023/05/1._read_more_books2._get_a_new_hobby3._try_rock_climbing4._be_more_creative_1__1.jpg]"
164,164,ddc70830-7d7c-11ef-acef-7295f59fdab3,9,API Orthodontic Pliers...,₹790.00,₹790.00,"[https://dentalstall.com/wp-content/uploads/2023/05/plier-300x300.jpg, https://dentalstall.com/wp-content/uploads/2023/05/plier-100x100.jpg, https://dentalstall.com/wp-content/uploads/2023/05/plier-150x150.jpg, https://dentalstall.com/wp-content/uploads/2023

In [483]:
prod_metadata_df.head(50)

,product_identifier,page_id,product_description,published_price,discounted_price,linked_images
0,24e0001a-7d7c-11ef-acef-7295f59fdab3,2,1 x GDC Extraction Forceps Lo...,₹1250.00,₹850.00,"[https://dentalstall.com/wp-content/uploads/2021/11/GDC-Extraction-Forceps-Lower-Molars-86A-Standard-FX86AS-300x300.jpg, https://dentalstall.com/wp-content/uploads/2021/11/GDC-Extraction-Forceps-Lower-Molars-86A-Standard-FX86AS-100x100.jpg]"
1,24e018d4-7d7c-11ef-acef-7295f59fdab3,2,3A MEDES Bleaching And Night ...,₹1380.00,₹1380.00,"[https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1-300x300.jpg, https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1-100x100.jpg, https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1-600x600.jpg, https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1-150x150.jpg, https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1-768x768.jpg, https://dentalstall.com/wp-content/uploads/2023/02/3a-medes-bleaching-and-night-guard-sheets-2-1.jpg]"
2,24e02f36-7d7c-11ef-acef-7295f59fdab3,2,3m Espe Adper Easy Bond Self-...,₹2863.00,₹2365.00,"[https://dentalstall.com/wp-content/uploads/2023/03/3mespe082010-300x300.jpg, https://dentalstall.com/wp-content/uploads/2023/03/3mespe082010-100x100.jpg, https://dentalstall.com/wp-content/uploads/2023/03/3mespe082010-150x150.jpg, https://dentalstall.com/wp-content/uploads/2023/03/3mespe082010.jpg]"
3,24e04a3e-7d7c-11ef-acef-7295f59fdab3,2,3M ESPE Adper Scotchbond Mult...,₹1195.00,₹1195.00,"[https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-300x300.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-100x100.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-600x600.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-1024x1024.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-150x150.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7-768x768.jpg, https://dentalstall.com/wp-content/uploads/2023/01/artboard_1_copy_7.jpg]"
4,24e05c54-7d7c-11ef-acef-7295f59fdab3,2,3m Espe Adper Single Bond Uni...,₹2590.00,₹2590.00,"[https://dentalstall.com/wp-content/uploads/2021/12/3m-Espe-Adper-Single-Bond-Universal-Adhesive-300x300.jpg, https://dentalstall.com/wp-content/uploads/2021/12/3m-Espe-Adper-Single-Bond-Universal-Adhesive-100x100.jpg, https://dentalstall.com/wp-content/uploads/2021/12/3m-Espe-Adper-Single-Bond-Universal-Adhesive.jpg]"
5,24e06eec-7d7c-11ef-acef-7295f59fdab3,2,3m Espe Clinpro Sealant ̵...,₹1995.00,₹1995.00,"[https://dentalstall.com/wp-content/uploads/2023/03/65e2c135c8ee024c58e0422947cbbfc0-1-300x300.jpg, https://dentalstall.com/wp-content/uploads/2023/03/65e2c135c8ee024c58e0422947cbbfc0-1-100x100.jpg, https://dentalstall.com/wp-content/uploads/2023/03/65e2c135c8ee024c58e0422947cbbfc0-1-150x150.jpg]"
6,24e07f2c-7d7c-11ef-acef-7295f59fdab3,2,3M ESPE Clinpro Tooth Creme...,₹500.00,₹500.00,"[https://dentalstall.com/wp-content/uploads/2021/12/3m-Espe-Clinpro-Tooth-Creme.-3-300x300.jpg, https://dentalstall.com/wp-content/uploads/2021/12/3m-Espe-Clinpro-Tooth-Creme.-3-100x100.jpg]"
7,24e09106-7d7c-11ef-acef-7295f59fdab3,2,3M Espe Composite Syringe Org...,₹2990.00,₹995.00,"[https://dentalstall.com/wp-content/uploads/2021/12/61iV6qF-bL._SL1000_-300x300.jpg, https://dentalstall.com/wp-content/uploads/2021/12/61iV6qF-bL._SL1000_-100x100.jpg, https://dentalstall.com/wp-content/uploads/2021/12/61iV6qF-bL._SL1000_-600x600.jpg, https://dentalstall.com/wp-content/uploads/2021/12/61iV6qF-bL._SL1000_-150x150.jpg, https://dentalstall.com/wp-content/uploads/2021/12/61iV6qF-bL._SL1000_-768x768.jpg, https://dentalstall.com/wp-content/uploads/2021/12/61iV6qF-bL._SL1000_.jpg]"
8,24e0a10a-7d7c-11ef-acef-7295f59fdab3,2,3m Espe Elipar Deepcure Led &...,₹82966.00,₹54600.00

In [411]:
import re

_str = "https://dentalstall.com/wp-content/uploads/2021/11/GDC-Extraction-Forceps-Lower-Molars-86A-Standard-FX86AS-300x300.jpg 300w"

regex = re.search(r"\s*\d+w$", _str)
print(regex[0])

_str.split("{}".format(regex[0]))


 300w


['https://dentalstall.com/wp-content/uploads/2021/11/GDC-Extraction-Forceps-Lower-Molars-86A-Standard-FX86AS-300x300.jpg',
 '']